In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functions import *

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor, make_column_selector

# 2. Model selection

We will apply the lessons learned during the data exploration to our dataset, before looking into potential regression algorithms for our specific problem.

In [7]:
diam_data = pd.read_csv('diamonds.csv', index_col = 0)
diam_data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Our ultimate target is the price, which will be computed as $price/carat \times weight$. For our model, the target will however be the price/carat.

In [8]:
y = np.divide(diam_data['price'], diam_data['carat'])
y.head()

1    1417.391304
2    1552.380952
3    1421.739130
4    1151.724138
5    1080.645161
dtype: float64

In [9]:
X = diam_data.drop(columns=['price'])
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


We know a step is the log-transformation of our target variable.

In [10]:
target_transform = TransformedTargetRegressor(
    regressor=None, #To be set for each model
    func=np.log, #Use log-transform
    inverse_func=np.exp #Inverse = exponential
)

Another step includes binning the 'carat' column, before using it in the model.

Scikit-learn does not have a simple function to achieve this with given bin edges, so we will create one that can be used in a pipeline.

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

In [12]:
class CustomBinDiscretizer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 bins,
                 right: 'bool' = True,
                 labels=None,
                 retbins: 'bool' = False,
                 precision: 'int' = 3,
                 include_lowest: 'bool' = False,
                 duplicates: 'str' = 'raise',
                 ordered: 'bool' = True):
        self.bins = bins
        self.right = right
        self.labels = labels
        self.retbins = retbins
        self.precision = precision
        self.include_lowest = include_lowest
        self.duplicates = duplicates
        self.ordered = ordered

    def fit(self, X, y=None):
        #Nothing to fit, given custom bins
        return self

    def transform(self, X, y=None):
        # if isinstance(X, pd.DataFrame), ("Only pandas dataframes can be used as inputs for this function")
        X_new = pd.DataFrame(X)
        for col in X_new.columns:
            X_new.loc[:, col] = pd.cut(x=X_new.loc[:, col].values, **self.__dict__)
        return X_new

We then initialize this custom class with the relevant parameters.

In [13]:
discretizer = CustomBinDiscretizer(
    labels=False, #No labels, using int ordinal instead directly
    bins=[0, .5, .75, 1, 1.5, 2, +np.inf]
)

Another step involves the imputation, through bayesian ridge regression, of the missing x values, if any.

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [15]:
#Imputation step
imputer = IterativeImputer(missing_values=0, random_state=50)

Depending on the algorithm used, we may also need additional steps:
- Encoding (ordinally, or one-hot) categorical variables
- Standard-scaling numerical values

In [16]:
#Define ordinal order for the encoded categorical variables
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [17]:
#Initialize encoders
oh_encoder = OneHotEncoder()
or_encoder = OrdinalEncoder(categories = [cut_order, clarity_order])

In [18]:
#Initialize standard scaler
scaler = StandardScaler()

In [19]:
#Preprocessing steps for columns used in imputer
preprocess_imputer = ColumnTransformer([
    ('discretizer', discretizer, [0]), #Using lists of scalar as column position, as the name gets removed after the first step (imputer)
    ('scaler', scaler, [1])
])

#Combining the two
prep_imputer = Pipeline([
    ('imputer', imputer),
    ('preprocess_imputer', preprocess_imputer)
])

#Overall preprocessing
preprocess = ColumnTransformer(
    [
        ('prep_imputer', prep_imputer, ['carat', 'x']),
        ('oh_encoder', oh_encoder, ['color']),
        ('or_encoder', or_encoder, ['cut', 'clarity']),
        ('scaler', scaler, ['depth', 'table'])
    ],
    remainder='drop'
)

## Models

We can now start looking at potential predictive models based on our dataset.
We will use 80% of the data for training purposes (of which 90% for direct training, 10% for model selection / validation), and the remaining 20% as a test set.
The models will be fit with 5-fold cross-validation on the direct training set, using the mean squared error as the main loss function, and a combination of factors will be used for model selection.

In [20]:
#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=50)

#Train-validation split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=5)

In [21]:
from sklearn.linear_model import SGDRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor

### Stochastic Gradient Descent regressor with elasticnet penalty

In [22]:
#Define the model
sgdr_model = SGDRegressor(
    loss='squared_error',
    penalty='elasticnet',
    random_state=50
)

In [23]:
#Create a pipeline with preprocessing + model
sgdr_pipe = Pipeline([
    ('preprocess', preprocess),
    ('model', sgdr_model)
])

In [24]:
sgdr_pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('prep_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   IterativeImputer(missing_values=0,
                                                                                    random_state=50)),
                                                                  ('preprocess_imputer',
                                                                   ColumnTransformer(transformers=[('discretizer',
                                                                                                    CustomBinDiscretizer(bins=[0,
                                                                                                                               0.5,
                                                                                                                               0.75,
                                                                                                                               1,
                                                                                                                               1.5,
                                                                                                                               2,
                                                                                                                               inf],
                                                                                                                         labels=False),
                                                                                                    [0]),
                                                                                                   ('scaler',
                                                                                                    StandardScaler(),
                                                                                                    [1])]))]),
                                                  ['carat', 'x']),
                                                 ('oh_encoder', OneHotEncoder(),
                                                  ['color']),
                                                 ('or_encoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']]),
                                                  ['cut', 'clarity']),
                                                 ('scaler', StandardScaler(),
                                                  ['depth', 'table'])])),
                ('model', SGDRegressor(penalty='elasticnet', random_state=50))])

In [25]:
#Include the log-transformation of the target into the model
sgdr_pipe_transform = TransformedTargetRegressor(
    regressor=sgdr_pipe,
    func=np.log, #Use log-transform
    inverse_func=np.exp #Inverse = exponential
)

In [26]:
#Define parameters grid for the grid search
grid = {
    'regressor__model__alpha': np.logspace(-5,0,6),
    'regressor__model__l1_ratio': np.linspace(0.1,0.9,5)
}

In [27]:
#Include the model into the grid search
sgdr_pipe_transform = GridSearchCV(
    sgdr_pipe_transform,
    param_grid=grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=10
)

In [28]:
sgdr_pipe_transform.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=TransformedTargetRegressor(func=<ufunc 'log'>,
                                                  inverse_func=<ufunc 'exp'>,
                                                  regressor=Pipeline(steps=[('preprocess',
                                                                             ColumnTransformer(transformers=[('prep_imputer',
                                                                                                              Pipeline(steps=[('imputer',
                                                                                                                               IterativeImputer(missing_values=0,
                                                                                                                                                random_state=50)),
                                                                                                                              ('preprocess_imputer',
                                                                                                                               ColumnTransformer(transformers=[('discretizer',
                                                                                                                                                                CustomBinDiscre...
                                                                                                                                          'VS1',
                                                                                                                                          'VVS2',
                                                                                                                                          'VVS1',
                                                                                                                                          'IF']]),
                                                                                                              ['cut',
                                                                                                               'clarity']),
                                                                                                             ('scaler',
                                                                                                              StandardScaler(),
                                                                                                              ['depth',
                                                                                                               'table'])])),
                                                                            ('model',
                                                                             SGDRegressor(penalty='elasticnet',
                                                                                          random_state=50))])),
             n_jobs=10,
             param_grid={'regressor__model__alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]),
                         'regressor__model__l1_ratio': array([0.1, 0.3, 0.5, 0.7, 0.9])},
             scoring='neg_mean_squared_error')

### Partial Least Squares regressor

In [29]:
#Define the model
plsr_model = PLSRegression(
    scale=False
)

In [30]:
#Create a pipeline with preprocessing + model
plsr_pipe = Pipeline([
    ('preprocess', preprocess),
    ('model', plsr_model)
])

In [31]:
plsr_pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('prep_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   IterativeImputer(missing_values=0,
                                                                                    random_state=50)),
                                                                  ('preprocess_imputer',
                                                                   ColumnTransformer(transformers=[('discretizer',
                                                                                                    CustomBinDiscretizer(bins=[0,
                                                                                                                               0.5,
                                                                                                                               0.75,
                                                                                                                               1,
                                                                                                                               1.5,
                                                                                                                               2,
                                                                                                                               inf],
                                                                                                                         labels=False),
                                                                                                    [0]),
                                                                                                   ('scaler',
                                                                                                    StandardScaler(),
                                                                                                    [1])]))]),
                                                  ['carat', 'x']),
                                                 ('oh_encoder', OneHotEncoder(),
                                                  ['color']),
                                                 ('or_encoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']]),
                                                  ['cut', 'clarity']),
                                                 ('scaler', StandardScaler(),
                                                  ['depth', 'table'])])),
                ('model', PLSRegression(scale=False))])

In [32]:
#Include the log-transformation of the target into the model
plsr_pipe_transform = TransformedTargetRegressor(
    regressor=plsr_pipe,
    func=np.log, #Use log-transform
    inverse_func=np.exp #Inverse = exponential
)

In [33]:
#Define parameters grid for the grid search
grid = {
    'regressor__model__n_components': [2,4,6,8],
}

In [34]:
#Include the model into the grid search
plsr_pipe_transform = GridSearchCV(
    plsr_pipe_transform,
    param_grid=grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=10
)

In [35]:
plsr_pipe_transform.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=TransformedTargetRegressor(func=<ufunc 'log'>,
                                                  inverse_func=<ufunc 'exp'>,
                                                  regressor=Pipeline(steps=[('preprocess',
                                                                             ColumnTransformer(transformers=[('prep_imputer',
                                                                                                              Pipeline(steps=[('imputer',
                                                                                                                               IterativeImputer(missing_values=0,
                                                                                                                                                random_state=50)),
                                                                                                                              ('preprocess_imputer',
                                                                                                                               ColumnTransformer(transformers=[('discretizer',
                                                                                                                                                                CustomBinDiscre...
                                                                                                              ['color']),
                                                                                                             ('or_encoder',
                                                                                                              OrdinalEncoder(categories=[['Fair',
                                                                                                                                          'Good',
                                                                                                                                          'Very '
                                                                                                                                          'Good',
                                                                                                                                          'Premium',
                                                                                                                                          'Ideal'],
                                                                                                                                         ['I1',
                                                                                                                                          'SI2',
                                                                                                                                          'SI1',
                                                                                                                                          'VS2',
                                                                                                                                          'VS1',
                                                                                                                                          'VVS2',
                                                                                                                                          'VVS1',
                                                                                                                                          'IF']]),
                                                                                                              ['cut',
                                                                                                               'clarity']),
                                                                                                             ('scaler'

### KNN regressor

In [26]:
knnr_model = KNeighborsRegressor(
    n_jobs=10
)

### Light GBM regressor

In [27]:
lgbr_model = LGBMRegressor(
    n_jobs=10,
    random_state=50
)